# Coleta de dados
A fim de criar um modelo para classificar os fornecedores do Governo Federal entre os que tem alto risco de descumprimento contratual e os de baixo risco, precisaremos obter os seguintes dados:
* _Contratos_ firmados por empresas com o Governo Federal, disponíveis no [Portal da Transparência do Governo Federal](http://www.portaltransparencia.gov.br/download-de-dados/compras)
* _Ocorrências por fornecedor_, disponíveis via [API do site Compras Governamentais](http://compras.dados.gov.br/docs/fornecedores/v1/ocorrencias_fornecedores.html)
* _Licitações realizadas_ pelo Governo Federal, com dados sobre os participantes e vencedores, disponíveis no [Portal da Transparência do Governo Federal](https://www.portaltransparencia.gov.br/download-de-dados/licitacoes)
* _Dados cadastrais dos fornecedores_, disponíveis na [base do CNPJ, já tratada](https://github.com/georgevbsantiago/qsacnpj)

In [1]:
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.9


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import grequests
import requests
from bs4 import BeautifulSoup

## Ocorrências por fornecedor
Para obter as ocorrências por fornecedor, utilizamos a [API do site Compras Governamentais](http://compras.dados.gov.br/docs/fornecedores/v1/ocorrencias_fornecedores.html). Os dados brutos serão salvos no arquivo _ocorrencias.csv_.

In [4]:
i = 0
n = 20 #número de requisições simultâneas
status = 200
cabecalho = ''
with open('./ocorrencias.csv', 'a') as outfile:
    while status == 200:
        urls = ['http://compras.dados.gov.br/fornecedores/v1/ocorrencias_fornecedores.csv?offset=' + str(500*j) for j in range(i, i+n)]
        requisicoes = (grequests.get(url) for url in urls)
        respostas = grequests.map(requisicoes)
        for resposta in respostas:
            status = resposta.status_code
            if status == 200:
                resposta.encoding = 'utf-8'
                if cabecalho == '':
                    cabecalho = resposta.text.split('\n')[0]
                    outfile.write(cabecalho + '\n')
                outfile.write(resposta.text.replace(cabecalho + '\n', '').replace(u'\u200b', ''))                
            else:
                url = urls[respostas.index(resposta)]
                ultimaPagina = requests.get(url.replace('.csv', '.html'))
                soup = BeautifulSoup(ultimaPagina.content, 'html.parser')
                print(soup.find('p').get_text())                
                break
        i += n

Intervalo de registros solicitado (acima de 260500) é maior que o total existente (260012 registros)


## Obtenção dos dados das licitações
A fonte destes dados é o [Portal da Transparência do Governo Federal](http://www.portaltransparencia.gov.br/download-de-dados/licitacoes). No momento da consulta, estavam disponíveis os dados das licitações de janeiro de 2013 a fevereiro de 2021. Os dados brutos serão salvos na pasta _licitacoes_, sendo um arquivo para cada mês. O dicionário de dados pode ser obtido [aqui](http://www.portaltransparencia.gov.br/pagina-interna/603389-dicionario-de-dados-licitacoes).

In [5]:
anos = [ano for ano in range(2013, 2022)]
meses = [mes for mes in range(1, 13)]

In [6]:
for ano in anos:
    for mes in meses:
        url = 'http://www.portaltransparencia.gov.br/download-de-dados/licitacoes/' + str(ano) + str(mes).zfill(2)
        r = requests.get(url, stream = True)
        with open('./licitacoes/' + str(ano) + str(mes).zfill(2) + '.zip', 'wb') as file:
            for chunk in r.iter_content(chunk_size = 128):
                file.write(chunk)
        if ano == 2021 and mes == 2:
            break

## Obtenção dos dados dos contratos
A fonte destes dados é o [Portal da Transparência do Governo Federal](http://www.portaltransparencia.gov.br/download-de-dados/compras). No momento da consulta, estavam disponíveis os dados dos contratos de janeiro de 2013 a março de 2021. Os dados brutos serão salvos na pasta _contratos_, sendo um arquivo para cada mês. O dicionário de dados pode ser obtido [aqui](http://www.portaltransparencia.gov.br/pagina-interna/603396-dicionario-de-dados-contratos-firmados).

In [8]:
for ano in anos:
    for mes in meses:
        url = 'http://www.portaltransparencia.gov.br/download-de-dados/compras/' + str(ano) + str(mes).zfill(2)
        r = requests.get(url, stream = True)
        with open('./contratos/' + str(ano) + str(mes).zfill(2) + '.zip', 'wb') as file:
            for chunk in r.iter_content(chunk_size = 128):
                file.write(chunk)
        if ano == 2021 and mes == 3:
            break

## Obtenção dos dados do cadastro do CNPJ
Utilizaremos a base do cadastro de CNPJ que a Receita Federal disponibiliza [neste link](http://receita.economia.gov.br/orientacao/tributaria/cadastros/cadastro-nacional-de-pessoas-juridicas-cnpj/dados-publicos-cnpj), juntamente com o [layout](http://200.152.38.155/CNPJ/LAYOUT_DADOS_ABERTOS_CNPJ.pdf). Porém, esta base como disponibilizada no site da RFB está num formato bem específico, é muito grande e contém mais dados do que o que vamos precisar. [Neste outro link](https://github.com/georgevbsantiago/qsacnpj) podemos obter a mesma base, completa e em formato SQLite, o que vai facilitar bastante a utilização. Também é disponibilizado o [modelo lógico do banco de dados](https://raw.githubusercontent.com/georgevbsantiago/qsacnpj/master/img/esquema_cnpj.png). Vamos utilizar a base de dados liberada pela RFB no dia 23/11/2020 e processada com a versão 0.2.2 do pacote _qsacnpj_, que é a versão mais recente disponível para download no momento da elaboração desta análise. Este download foi feito manualmente, acessando o link via navegador e baixando o arquivo _sqlite_dados_qsa_cnpj_23-11-20.zip_.